# Retrieve Accreditations Script
This script is used to retrieve the authors, title, date, and arXiv identifier of the paper that an image is taken from.

In [ ]:
import sqlite3
import re

In [ ]:
db_path = "/home/rte/data/db/arxiv_db_images.sqlite3"

In [ ]:
# Here we import the sqlite3 database and create a cursor

db = sqlite3.connect(db_path)
c = db.cursor()

In [ ]:
c.execute('PRAGMA TABLE_INFO({})'.format("metadata"))
info = c.fetchall()

print("\nColumn Info:\nID, Name, Type, NotNull, DefaultVal, PrimaryKey")
for col in info:
    print(col)

In [ ]:
c.execute('PRAGMA TABLE_INFO({})'.format("images"))
info = c.fetchall()

print("\nColumn Info:\nID, Name, Type, NotNull, DefaultVal, PrimaryKey")
for col in info:
    print(col)

In [ ]:
sql = ('''
    SELECT metadata.id, metadata.identifier, metadata.title, metadata.abstract
    FROM metadata
    WHERE metadata.title LIKE '%arxiv%'
    AND metadata.abstract LIKE '%data%'
    AND metadata.title NOT LIKE '%reply%'
    AND metadata.title NOT LIKE '%comment%'
    AND metadata.title NOT LIKE '%addendum%'
    AND metadata.title NOT LIKE '%response%'
    AND metadata.title NOT LIKE '%remark%'
    ''')

c.execute(sql, )
rows = c.fetchall()

print(len(rows))
for row in rows:
    print(row)

In [ ]:
# example list of filepaths
file_ids = ['512.jpg', '171118.jpg', '55685.jpg', '41640.jpg', '161294.jpg', '119145.jpg', '94606.jpg', '12318.jpg', '81674.jpg', '135180.jpg', '166435.jpg', '88950.jpg', '116863.jpg', '113725.jpg', '100418.jpg', '10263.jpg', '107188.jpg', '6875.jpg', '159687.jpg', '121848.jpg', '81734.jpg', '39624.jpg', '153856.jpg', '154580.jpg', '133217.jpg', '149106.jpg', '5600.jpg', '161772.jpg', '125292.jpg', '145409.jpg', '9467.jpg', '169317.jpg', '16795.jpg', '19158.jpg', '34999.jpg', '162889.jpg', '158361.jpg', '19209.jpg', '31444.jpg', '159528.jpg', '41253.jpg', '19789.jpg', '138808.jpg', '22011.jpg', '41143.jpg', '133089.jpg', '108140.jpg', '134224.jpg', '11905.jpg', '16398.jpg', '7692.jpg', '90834.jpg', '19967.jpg', '121213.jpg', '124903.jpg', '18595.jpg', '63035.jpg', '60238.jpg', '17403.jpg', '161580.jpg', '127862.jpg', '18093.jpg', '27731.jpg', '133703.jpg', '150958.jpg', '88809.jpg', '33062.jpg', '22310.jpg', '135194.jpg', '64159.jpg', '64841.jpg', '19289.jpg', '164368.jpg', '156364.jpg', '5765.jpg', '161153.jpg', '170697.jpg', '62943.jpg', '27003.jpg', '147305.jpg', '57456.jpg', '43216.jpg', '80323.jpg', '100930.jpg', '108460.jpg', '111245.jpg', '44472.jpg', '172425.jpg', '124945.jpg', '102597.jpg', '139856.jpg', '153040.jpg', '6234.jpg', '45166.jpg', '49261.jpg', '129597.jpg', '36854.jpg', '77972.jpg', '38244.jpg', '56562.jpg']
print(file_ids[0])

In [ ]:
# cs.CV 2018-10 montage
file_ids = [6487608, 6397498, 6486928, 6378127, 6376090, 6455435, 6491014, 6422849, 6477672, 6452584, 6412993, 6495710, 6425938, 6460573, 6448827, 6444424, 6463722, 6397882, 6402381, 6398190, 6393234, 6469087, 6405727, 6460612, 6489566, 6457745, 6406389, 6402963, 6412055, 6391001, 6473542, 6388309, 6429963, 6473774, 6394635, 6388429, 6489893, 6371778, 6466761, 6446509, 6407169, 6472172, 6462027, 6386079, 6388367, 6438412, 6463857, 6397601, 6394633, 6449325, 6460723, 6419074, 6444422, 6397595, 6395451, 6402966, 6467888, 6446874, 6468760, 6423203, 6489722, 6462063, 6415187, 6382931, 6458833, 6390969, 6405612, 6444319, 6470426, 6489744, 6395480, 6444394, 6388229, 6418335, 6432722, 6391917, 6470442, 6434288, 6418322, 6403304, 6477353, 6374697, 6419026, 6386666, 6418358, 6462291, 6485481, 6455065, 6450234, 6458711, 6422733, 6458871, 6453167, 6405777, 6405591, 6469091, 6444317, 6388020, 6460639, 6425076, 6486898, 6477347, 6484143, 6463721, 6460310, 6463802, 6397275, 6489903, 6469010, 6423217, 6446389, 6397277, 6483270, 6489931, 593773, 6396421, 6378085, 6398017, 6449681, 6483282, 6422183, 6389548, 6405866, 6454964, 6442910, 6422799, 6481324, 568694, 6489544, 6382934, 6458706, 6388387, 6483209, 6473572, 6467425, 6397550, 6434291, 6490675, 6425098, 6391004, 6392694, 6409647, 6493136, 6410602]

In [ ]:
# random montage
file_ids = [6984415, 3125589, 7156363, 1274388, 1329777, 6116560, 177609, 3228276, 2539254, 5878409, 4197851, 3670232, 7910772, 2575360, 3342047, 4516356, 4473054, 818325, 3117143, 5769995, 1280294, 3717386, 4177356, 1378201, 8053592, 3286251, 8601314, 6356928, 9870325, 4695134, 8086512, 9889860, 136248, 8092998, 810922, 2498029, 8002827, 7297084, 9583876, 9370318, 3254892, 3241273, 2140483, 364274, 4960236, 1161724, 3620891, 1756395, 6649210, 6146247, 7999839, 7156106, 9035548, 9070051, 9758377, 3642704, 1476858, 3556222, 3063502, 56419, 8102212, 3402711, 5559379, 3984650, 2155519, 9263207, 7261028, 966164, 2238214, 9549350, 8895530, 2010682, 1638030, 9150589, 7950484, 2116321, 2010771, 3688046, 6440060, 5011940, 5120862, 3497617, 6061730, 1194670, 4023986, 2872354, 9906679, 2620366, 9493768, 2453532, 2139889, 1738360, 10041136, 5875249, 7182058, 8638012, 7588299, 4226578, 1621699, 5500682, 6101706, 2498675, 7839453, 4480272, 7057687, 4037991, 7264942, 3186375, 5207065, 6025709, 2116847, 2359172, 7432775, 5909122, 8589970, 4662673, 2888300, 4414427, 7221250, 394633, 2217591, 486305, 2939467, 4471869, 4565378, 9644692, 7290021, 5045554, 2046788, 6164160, 3460546, 5254068, 6964891, 5018194, 6114985, 7658549, 4379701, 6545909, 9373974, 2994704, 3013192, 7259556, 2450294, 3608380]

In [ ]:
# 4x4 random montage take 1
file_ids = [4727821, 6477578, 8235637, 2908964, 6667466, 9647394, 9074544, 3596656, 304611, 1728181, 523743, 3188878, 6815751, 3327396, 107870, 5108768]

In [ ]:
# random montage 4x4 v2
file_ids = [7537493, 9294444, 9532468, 9188103, 2625381, 8713581, 4774848, 5181615, 5800544, 7832325, 8317765, 4057245, 7066761, 6090763, 2573732, 4094796]

In [ ]:
# stat.ML 12x12
file_ids = [3741700, 6792861, 9893600, 1152413, 5375948, 9307123, 6719971, 3081421, 2307475, 6481584, 1451101, 462258, 2524846, 6455825, 7839970, 3652374, 5161177, 8669975, 7831481, 761781, 8548743, 8100309, 9588924, 5115013, 404569, 7836336, 6417956, 6574662, 7359468, 4104550, 6594165, 8577244, 337244, 9970244, 7817836, 8048548, 7836185, 2393938, 7850941, 2307719, 4133575, 1066947, 507145, 7834026, 1004756, 5136125, 8512879, 5136127, 7807160, 7118290, 357495, 5757304, 4133570, 4123516, 1196654, 1024189, 4744273, 2525181, 4114341, 790137, 9603947, 2496748, 6652521, 8711872, 4076959, 7835353, 4054178, 5555666, 4660369, 3085746, 2337423, 2552367, 6438543, 3368684, 4698575, 10008883, 7834809, 3756352, 99852, 1969188, 9883699, 8476189, 4772102, 1153925, 583799, 533421, 1672433, 2307768, 5106479, 1766009, 8494791, 6486821, 4055653, 376783, 4318405, 446488, 9588638, 1008135, 399385, 3481477, 7579994, 4082062, 7833865, 6841159, 9141734, 6786198, 6498479, 6571574, 548587, 7827295, 4351524, 4481669, 9932149, 6438719, 6602613, 2559488, 306609, 1957385, 6574918, 7830713, 7827361, 284318, 574221, 195940, 6114983, 2479948, 7702077, 6301433, 7833274, 4496743, 4682120, 4294710, 4671961, 8494128, 507247, 6056661, 3747640, 2541322, 499995, 8506109, 2359466, 2925185, 7832858, 1404810]

In [ ]:
# table showing image metadata
file_ids = [4876126, 2209549]

In [ ]:
# table showing article metadata
file_ids = ['0912.5313', '1308.0124']

## Build list of images

In [ ]:
# Get 144 images from stat.ML

sql = ("SELECT metadata.cat, images.path, images.filename, images.identifier, metadata.created, images.id "
    "FROM images "
    "LEFT JOIN metadata ON images.identifier = metadata.identifier "
    "WHERE images.x != '' "
    "AND substr(trim(metadata.cat),1,instr(trim(metadata.cat)||' ',' ')-1) = ? "
    "ORDER BY RANDOM() "
    "LIMIT 144 ")

c.execute(sql, ('stat.ML', ))
rows = c.fetchall()

In [ ]:
print(len(rows))
for row in rows:
    print(row)

In [ ]:
file_ids = []
for row in rows:
    file_ids.append(row[3])

In [ ]:
print(len(file_ids))
print(file_ids)

In [ ]:
# 144 images from stat.ML
file_ids = ['1707.07821', '1504.02338', '1512.08643', '1804.08646', '1710.07400', '1402.4624', '1806.03207', '1602.04601', '1310.6740', '1712.09988', '1810.11630', '1409.0031', '1711.02282', '1606.07129', '1711.06464', '1509.01173', '1503.01401', '1606.00972', '1809.03461', '1704.08265', '1502.02860', '1112.0611', '1606.00972', '1304.6803', '1412.7012', '1812.01161', '1310.4456', '1402.4624', '1712.02311', '1805.08656', '1209.4690', '1606.00972', '1507.02284', '1705.00740', '1401.0942', '1307.2302', '1811.01179', '1505.05004', '1606.00972', '1606.00972', '1609.04388', '1605.07174', '1606.00972', '1612.03957', '1811.00974', '1709.05501', '1606.00972', '1602.00236', '1606.00972', '1606.00972', '1606.00972', '1307.0293', '1710.11469', '1802.01396', '1609.00577', '1606.00972', '1608.02549', '1605.03027', '1506.08002', '1303.3664', '1710.11469', '1310.2880', '1606.00972', '1707.01164', '1706.07561', '1607.03300', '1810.07151', '1502.02860', '1801.05558', '1606.00972', '1710.05279', '1606.05596', '1802.07079', '1509.08588', '1409.7495', '1707.09241', '1606.00972', '1509.04210', '1402.4304', '1705.10941', '1606.00972', '1706.02496', '1611.08568', '1807.09306', '1001.0279', '1511.06481', '1606.00972', '1704.06625', '1605.07066', '1206.1088', '1606.00972', '1810.09184', '1706.04987', '1803.05288', '1805.02590', '1705.10102', '1803.08700', '1507.00996', '1311.2694', '1711.07076', '1606.00136', '1803.08882', '1808.01126', '1802.10497', '1610.00168', '1107.3133', '1708.01440', '1505.05004', '1710.09554', '1708.05715', '1612.05678', '1606.00972', '1309.1369', '1605.05697', '1607.06364', '1507.00996', '1511.08136', '1709.03423', '1708.00049', '1708.01955', '1606.00972', '1805.07438', '1510.08370', '1805.08340', '1301.3468', '1805.09874', '1505.07067', '1808.02433', '1606.00972', '1802.07079', '1811.09757', '1702.07398', '1606.00972', '1708.07967', '1808.08317', '1506.03768', '1810.00597', '1711.00799', '1812.10587', '1802.04065', '1311.1354', '1207.2812', '1202.0825', '1505.02065']

## Format file_ids for sql processing

Shouldn't need to do this anymore, actually

In [ ]:
# rewrite file_ids as strings
file_ids = [str(x) for x in file_ids[:]]

In [ ]:
# print the list, useful for copying over to javascript ^_^
print(file_ids)

In [ ]:
print(len(file_ids))

### if filenames with extension

In [ ]:
# get id from filename
for i, _ in enumerate(file_ids):
    # grab everything before the first "."
    file_ids[i] = (file_ids[i].split(".")[0])
print(file_ids)

### if identifiers only

In [ ]:
# or just convert to tuple
file_ids = [(x,) for x in file_ids[:]]

In [ ]:
print(file_ids)

## Query SQL and format
Here we construct an SQLite query that allows us to join the ```metadata``` and ```images``` tables. We get the author, title, date, and identifer. We then use this to format an accreditation that can be used for papers or websites. 

In [ ]:
# search for a specific set of entries and retrieve metadata
# this is for the image identifier

rows = []

sql = ("SELECT metadata.authors, metadata.title, metadata.created, images.identifier "
    "FROM images "
    "LEFT JOIN metadata ON images.identifier = metadata.identifier "
    "WHERE images.id is ? ")

# iterate over the file_ids list and grab sql data
for file_id in file_ids[:]:
    print("image parent article id:",file_id)
    c.execute(sql, (file_id))
    row = c.fetchall()
    rows.append(row)
    print(row)
    print(len(row))
    print(len(row[0]))
#     print(list(rows))


In [ ]:
# search ARTICLES IDENTIFIER

rows = []

sql = ("SELECT metadata.authors, metadata.title, metadata.created, metadata.identifier "
    "FROM metadata "
    "WHERE metadata.identifier is ? ")

# iterate over the file_ids list and grab sql data
for file_id in file_ids[:]:
    print("article id:",file_id)
    c.execute(sql, (file_id))
    row = c.fetchall()
    rows.append(row)
    print(row)
    print(len(row))
    print(len(row[0]))
#     print(list(rows))


In [ ]:
print(len(rows))

In [ ]:
# go through all of the retrieved SQL rows and format as an accreditation
# if html is True, format with <a href=####> for web usage

accreditations = []

# html = True
html = False

# grab the details and check each
for row in rows:
    s = row[0][0]
    start = "['"
    end = "']"
    author = s[s.find(start)+len(start):s.rfind(end)]
    author = author[:-2]
    print(author)
    
    title = row[0][1]
    # replace line breaks and double spaces
    title = title.replace("\n", "").replace("  "," ")
    print(title)
    
    date = row[0][2].split("-")[0]
    print(date)
    
    identifier = row[0][3]
    print(identifier)
#     reg_exp = re.compile("/[^\d]\d{2}[^\d]/")
    longest_digits = max(re.findall(r'\d+', identifier), key = len)
    print(len(longest_digits))
    
    # if the identifier contains seven consecutive numbers, add a slash
    if len(longest_digits) == 7:
        print("----- regex match -----")
        reverse = identifier[::-1]
        print(reverse)
        identifier_reverse = reverse[:7] + "/" + reverse[7:]
        identifier = identifier_reverse[::-1]
    else:
        # otherwise we can leave the identifier how it is
        print("----- no match -----")
    print(identifier)
    url = "https://arxiv.org/abs/" + identifier
    print(url)
#     print(row[0][0])

    # format string and append
    if html:
        fmt_str = '{}: {}, {}, <a href="{}">{}</a>'
        accreditations.append(fmt_str.format(author, title, str(date), url, url))  
    else:
        fmt_str = '{}: {}, {}, {}'
        accreditations.append(fmt_str.format(author, title, str(date), url, url))    

    # harder way of doing it
#     accreditations.append([author + ': "' + title + '", ' + str(date) + ", " + '<a href="' + url + '">' + url + '</a>'])    


In [ ]:
# print out with line breaks
for row in accreditations:
    print(row)

In [ ]:
print(len(accreditations))

In [ ]:
print(accreditations)

In [ ]:
print(accreditations[0])

### Inverse: Find image credits from IDs

WIP

In [ ]:
# search for a specific set of entries and retrieve metadata
# this is for the image identifier

rows = []

sql = ("SELECT metadata.cat, images.path, images.filename, images.identifier, metadata.created, images.id "
    "FROM images "
    "LEFT JOIN metadata ON images.identifier = metadata.identifier "
    "WHERE images.id is ? ")

# iterate over the file_ids list and grab sql data
for file_id in file_ids[:]:
    print("image parent article id:",file_id)
    c.execute(sql, (file_id, ))
    row = c.fetchall()
    rows.append(row)
    print(row)
    print(len(row))
    print(len(row[0]))
#     print(list(rows))


In [ ]:
for row in rows:
    print(row)